In [1]:
import os
import random 
import numpy as np
import pandas as pd

import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import torch
print(torch.__version__)
import torchvision
print(torchvision.__version__)
from torchvision import transforms as torchtrans
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from engine import train_one_epoch, evaluate
import utils
import transforms as T

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

2.0.1+cu117
0.15.2+cu117


In [2]:
dataset_location = utils.load_rf_data(8)
dataset_location

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.153, to fix: `pip install ultralytics==8.0.134`


Extracting Dataset Version Zip to Football-Player-Detection-8 in yolov8:: 100%|██████████| 115/115 [00:00<00:00, 4412.92it/s]


'/home/mikkel/VSC/football-player-detection/Football-Player-Detection-8'

In [3]:
os.listdir(f"{dataset_location}/train/images")

['0b1495d3_1_mp4-6_jpg.rf.c22374d6e8a8cffe9d8f59eb778a88bb.jpg',
 '0b1495d3_0_mp4-8_jpg.rf.41d36e3782fbaea024a3768566c2eb99.jpg',
 '4dae79a9_0_mp4-1_jpg.rf.da1edbbd02e1a35afacbd37f243b0e1d.jpg',
 '2f54ed1c_0_mp4-0_jpg.rf.996817f8fc76090def995c9732615b57.jpg',
 '0b1495d3_0_mp4-0_jpg.rf.e50ef4d39b81adc248bda0b401c88fc4.jpg',
 '0b1495d3_0_mp4-0_jpg.rf.ca741cd9d337202959249e6cd18afdad.jpg',
 '9a70c54e_1_mp4-0_jpg.rf.952a64f5b62e78917a64f6b177599160.jpg',
 '0b1495d3_0_mp4-5_jpg.rf.3470dc2abf32b0d621229b88b6981627.jpg',
 '0b1495d3_1_mp4-0_jpg.rf.2503c68f752506af07e1202a93693b11.jpg',
 '0b1495d3_1_mp4-7_jpg.rf.1a1602e2cb4ccc1290b3e7a722e0cc2f.jpg',
 '0b1495d3_0_mp4-4_jpg.rf.7a860741f643727c09c3406712a64dc2.jpg',
 '0b1495d3_0_mp4-8_jpg.rf.363420f63286194974fda2e490cca99e.jpg',
 '4dae79a9_0_mp4-4_jpg.rf.18d0813adb387ab2e29c4a5bb868eafe.jpg',
 '9f4df856_0_mp4-2_jpg.rf.37ba88ec6f6ebe086fbdbeb7030cf918.jpg',
 '0b1495d3_0_mp4-1_jpg.rf.4972fa3d70659da3d041b34e99427ad2.jpg',
 '2f54ed1c_0_mp4-0_jpg.rf

In [ ]:
import yaml

class Dataset(torch.utils.data.Dataset):

    def __init__(self, dataset_location):
        with open(f"{dataset_location}/data.yaml") as f:
            doc = yaml.safe_load(f)
            self.img_dir = doc['train']+"/images"           
            self.label_dir = doc['train']+"/labels"
            self.classes = doc['names'].insert(0, 'background')

        self.imgs = [image for image in sorted(os.listdir(self.img_dir)) if image.endswith(".jpg")]
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]
    

dataset = Dataset(dataset_location)